# Build classification model: Sweet Asian and Indian Food


## Cuisine classifiers 2

For dis second lesson wey dey about classification, we go look `more ways` wey we fit take classify categorical data. We go still learn wetin fit happen if we choose one classifier instead of another one.

### [**Pre-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Prerequisite**

We dey assume say you don finish di previous lessons because we go dey use some concepts wey we don learn before.

For dis lesson, we go need di following packages:

-   `tidyverse`: Di [tidyverse](https://www.tidyverse.org/) na [collection of R packages](https://www.tidyverse.org/packages) wey dem design to make data science fast, easy and fun!

-   `tidymodels`: Di [tidymodels](https://www.tidymodels.org/) framework na [collection of packages](https://www.tidymodels.org/packages/) for modeling and machine learning.

-   `themis`: Di [themis package](https://themis.tidymodels.org/) dey provide Extra Recipes Steps to handle unbalanced data.

You fit install dem like dis:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Or, di script wey dey below go check whether you get di packages wey you need to complete dis module, and e go install dem for you if dem no dey.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

Make we start dey waka sharp sharp!

## **1. Map wey dey show classification**

For our [last lesson](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), we bin try answer dis question: how we go fit choose between plenty models? E dey depend well well on di kind data wey we get and di type problem wey we wan solve (like classification or regression?)

Before, we don learn about di different options wey dey available to classify data using Microsoft's cheat sheet. Python Machine Learning framework, Scikit-learn, get one cheat sheet wey dey more detailed wey fit help you narrow down your estimators (another name for classifiers):

<p >
   <img src="../../../../../../translated_images/pcm/map.e963a6a51349425a.webp"
   width="700"/>
   <figcaption></figcaption>


> Tip: [go check dis map online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) and click for di road to read di documentation.
>
> Di [Tidymodels reference site](https://www.tidymodels.org/find/parsnip/#models) sef get beta documentation about di different kain model.

### **Di plan** 🗺️

Dis map go help well well once you sabi your data well, as you fit 'waka' follow di road to make decision:

-   We get \>50 samples

-   We wan predict one category

-   We get labeled data

-   We get less than 100K samples

-   ✨ We fit choose Linear SVC

-   If e no work, since we get numeric data

    -   We fit try ✨ KNeighbors Classifier

        -   If e no work, try ✨ SVC and ✨ Ensemble Classifiers

Dis na very helpful road to follow. Now, make we jump enter am using di [tidymodels](https://www.tidymodels.org/) modelling framework: one consistent and flexible collection of R packages wey dem develop to encourage beta statistical practice 😊.

## 2. Share di data and handle imbalanced data set.

From di lesson wey we do before, we learn say some common ingredients dey across di cuisines. Plus, di number of cuisines no dey balance well.

We go handle dis one by:

-   Comot di most common ingredients wey dey cause wahala between di different cuisines, using `dplyr::select()`.

-   Use one `recipe` wey go preprocess di data to make am ready for modelling by applying one `over-sampling` algorithm.

We don already look dis one for di lesson wey we do before so e go easy well well 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### How to handle imbalanced data

Imbalanced data fit spoil model performance. Many models dey work well when di number of observations dey equal, so dem dey struggle when data no balance.

Two main ways dey to handle imbalanced data sets:

-   Add more observations to di minority class: `Over-sampling` e.g use SMOTE algorithm wey dey create new examples for di minority class by using nearest neighbors of di cases.

-   Remove some observations from di majority class: `Under-sampling`

For di previous lesson, we show how to handle imbalanced data sets using `recipe`. Recipe na like blueprint wey dey describe di steps wey you go follow for data set to prepare am for data analysis. For our case, we wan make di number of cuisines for our `training set` dey equal. Make we start.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Now we don ready to train models 👩‍💻👨‍💻!

## 3. Beyond multinomial regression models

For di last lesson, we bin look multinomial regression models. Make we check some models wey dey more flexible for classification.

### Support Vector Machines.

For classification matter, `Support Vector Machines` na one machine learning method wey dey try find one *hyperplane* wey go "best" separate di classes. Make we see one simple example:

<p >
   <img src="../../../../../../translated_images/pcm/svm.621ae7b516d678e0.webp"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ no dey separate di classes. H2~ dey separate dem, but e get small space. H3~ dey separate dem wit di biggest space.

#### Linear Support Vector Classifier

Support-Vector clustering (SVC) na one pikin for di Support-Vector machines family for ML techniques. For SVC, di hyperplane wey dem go choose go fit separate `most` of di training observations well, but e fit `misclassify` some observations. If dem allow some points dey for di wrong side, di SVM go strong pass for outliers and e go fit generalize well well for new data. Di parameter wey dey control dis violation na wetin dem dey call `cost` and e get default value of 1 (check `help("svm_poly")`).

Make we create linear SVC by setting `degree = 1` for polynomial SVM model.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Now wey we don capture di preprocessing steps and model specification inside one *workflow*, we fit go ahead train di linear SVC and check di results as we dey do am. For performance metrics, make we create one metric set wey go check: `accuracy`, `sensitivity`, `Positive Predicted Value` and `F Measure`

> `augment()` go add column(s) for predictions to di data wey dem give.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Support Vector Machine

Support vector machine (SVM) na extension of support vector classifier wey fit handle non-linear boundary between di classes. Di main idea be say SVMs dey use *kernel trick* to expand di feature space so e go fit work well with nonlinear relationships between di classes. One popular and very flexible kernel function wey SVMs dey use na *Radial basis function.* Make we see how e go perform for our data.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Much beta 🤩!

> ✅ Abeg check:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> for more tori.

### Nearest Neighbor classifiers

*K*-nearest neighbor (KNN) na one algorithm wey dey use similarity between observations to predict each one.

Make we fit am to our data.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

E be like say dis model no dey perform well. Maybe if you change di model argument dem (check `help("nearest_neighbor")`), e go fit make di model perform better. Make sure say you try am.

> ✅ Abeg check:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> to sabi more about *K*-Nearest Neighbors classifiers.

### Ensemble classifiers

Ensemble algorithm dem dey work by join plenty base estimator dem together to make one better model either by:

`bagging`: wey go use *averaging function* for di collection of base model dem

`boosting`: wey go build model dem one after di other to take improve di predictive performance.

Make we start by try Random Forest model, wey dey build plenty decision tree dem, come use averaging function join dem together to make di overall model better.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Good job 👏!

Make we try Boosted Tree model too.

Boosted Tree na one kind ensemble method wey dey build series of decision trees one after the other. Each tree go use the result of the tree wey dey before am to try reduce the error small small. E dey focus on the weight of items wey dem classify wrong and e go adjust the fit for the next classifier to correct am.

Different ways dey to fit this model (check `help("boost_tree")`). For this example, we go fit Boosted trees with `xgboost` engine.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Abeg check:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Dem dey talk about AdaBoost model wey fit work well as alternative to xgboost.
>
> to sabi more about Ensemble classifiers.

## 4. Extra - compare plenty models

We don fit plenty models for dis lab 🙌. E fit dey tire person or hard to dey create plenty workflows from different preprocessors and/or model specifications, come dey calculate performance metrics one by one.

Make we see if we fit solve dis mata by creating one function wey go fit list of workflows for training set, then e go return performance metrics based on test set. We go use `map()` and `map_dfr()` from the [purrr](https://purrr.tidyverse.org/) package to apply functions to each element for list.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) functions dey help you replace plenty for loops with code wey short and easy to read. The best place to sabi about [`map()`](https://purrr.tidyverse.org/reference/map.html) functions na the [iteration chapter](http://r4ds.had.co.nz/iteration.html) for R for data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

Make we call our function and check di accuracy for di models.


In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) package dey allow people create and fit plenty models easily, but e dey mostly work well with resampling techniques like `cross-validation`, wey we never talk about yet.

## **🚀Challenge**

Each of dis techniques get plenty parameters wey you fit adjust, like `cost` for SVMs, `neighbors` for KNN, `mtry` (Randomly Selected Predictors) for Random Forest.

Check how each one default parameters be and think about wetin go happen if you adjust these parameters for the model quality.

To sabi more about any model and e parameters, use: `help("model")` e.g `help("rand_forest")`

> For real life, we dey usually *estimate* the *best values* for these by training plenty models on top `simulated data set` and check how well all these models dey perform. Dis process na wetin dem dey call **tuning**.

### [**Post-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Review & Self Study**

Plenty big grammar dey for these lessons, so take small time review [dis list](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) of useful words!

#### THANK YOU TO:

[`Allison Horst`](https://twitter.com/allison_horst/) for the amazing drawings wey make R more friendly and fun. You fit see more of her drawings for her [gallery](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) and [Jen Looper](https://www.twitter.com/jenlooper) for creating the first Python version of dis module ♥️

Enjoy your learning,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../../../../../translated_images/pcm/r_learners_sm.f9199f76f1e2e493.webp"
   width="569"/>
   <figcaption>Artwork by @allison_horst</figcaption>


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI transleto service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translation. Even as we dey try make am correct, abeg sabi say machine translation fit get mistake or no dey accurate well. Di original dokyument wey dey for im native language na di main source wey you go trust. For important mata, e better make professional human transleto check am. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
